(EBs with 70-90% confidence value including 70 from Yifan's RF algorithm)
<br>
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [17]:
#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
#sector you want to classify
sector = 1

read in the prediction as well as the confidence values for each tic in sector 1, based off the RF model

In [3]:
sec1 = pd.read_csv('Sector1/sect1predandprob.csv', index_col=0)

In [ ]:
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint

In [29]:
data_dir = "/home/jupyter/mountpoint/"
ref = pd.read_csv(data_dir + 'sector1lookup.csv')
eb = sec1[sec1['probone'] > 0.7]
eb = eb[eb['probone'] < 0.9]

In [30]:
eb

,probzero,probone,pred
270221859,0.102000,0.898000,1
126803454,0.102000,0.898000,1
206488769,0.102000,0.898000,1
231702259,0.102000,0.898000,1
1989908229,0.102000,0.898000,1
...,...,...,...
325338582,0.298000,0.702000,1
99672412,0.298000,0.702000,1
211411347,0.298000,0.702000,1
358578246,0.298333,0.701667,1


In [46]:
tics = eb.index.values
len(tics)

553

In [ ]:
stars = ref.TIC_ID.isin(tics)
closestarslist = []
for i in range(len(tics)):
    rastar = ref[stars].RA.values[i]
    decstar = ref[stars].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist.append(closestars)
    
closestarstics = []
for i in range (len(closestarslist)):
    tics = []
    tics.append(ref[stars].TIC_ID.values[i])
    for j in range(len(closestarslist[i][0])):
        tics.append(ref[stars].TIC_ID.values[closestarslist[i][0][j]])
    closestarstics.append(tics)

In [33]:
refebs = ref.loc[ref['TIC_ID'].isin(eb.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [34]:
len(lcs)

553

In [35]:
tic = []
avgflux = []
    
for i in range(len(lcs)):
    tic.append(refebs['TIC_ID'].values[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [36]:
df = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(df)

In [37]:
highestfluxtics = []
for i in range(len(closestarstics)):
    a = []
    for j in closestarstics[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics[i][a.index(max(a))])

In [44]:
len(highestfluxtics)

553

In [38]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [39]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [40]:
newlist = [] #eb list with no duplicates, cut down to ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [41]:
len(newlist)

455